In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import time
import random
from tqdm import tqdm

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from PIL import Image

from sedensenet import densenet161

/home/usst/anaconda3/envs/dxzpy/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#1. set random.seed
import random 
seed = 34
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
use_gpu = True

In [3]:
data_path = "./"
train_path = data_path+'train/train/images/'
val_path = data_path+'val/val/images/'

train=pd.read_json(data_path+"/train.json")
print(len(train))
train.head()

31718


,disease_class,image_id
0,1,62fd8bf4d53a1b94fbac16738406f10b.jpg
1,1,0bdec5cccbcade6b6e94087cb5509d98.jpg
2,1,8951e940341f77c8d361c1872c67b16d.jpg
3,1,7ed158da58c451f75fb790530d6f19cc.jpg
4,1,9b7399aa-1c3c-4137-ae4e-196cd23fe573___FREC_Sc...


In [4]:
n_train = len(train)
categories = set(train['disease_class'])
n_class = len(categories)
print('disease_class:{}\n{}'.format(n_class,categories))

number_of_classes = []
for i in range(n_class):
    number_of_classes.append(list(train['disease_class']).count(i))
print(number_of_classes)

disease_class:61
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60}
[1185, 211, 152, 427, 142, 40, 598, 116, 110, 376, 191, 167, 483, 355, 208, 498, 815, 294, 381, 462, 503, 419, 61, 630, 367, 1828, 1799, 251, 857, 770, 1025, 287, 377, 1430, 203, 510, 251, 446, 242, 192, 583, 1208, 319, 966, 1, 1, 251, 442, 264, 1109, 325, 336, 43, 22, 421, 807, 542, 271, 1414, 2473, 261]


In [5]:
val = pd.read_json(data_path+"/val.json")
n_val = len(val)
number_of_classes = []
for i in range(n_class):
    number_of_classes.append(list(val['disease_class']).count(i))
print(number_of_classes)

[169, 30, 22, 61, 20, 6, 85, 12, 18, 54, 27, 24, 69, 51, 29, 71, 116, 42, 54, 66, 74, 59, 9, 90, 52, 269, 262, 36, 122, 110, 147, 40, 54, 204, 29, 73, 36, 64, 35, 27, 83, 173, 46, 138, 1, 0, 36, 63, 38, 158, 46, 48, 4, 5, 60, 115, 77, 39, 202, 353, 37]


In [6]:
num_classes = n_class
model = densenet161()

/media/usst/数据/DXZ/ai-challenger-pdr/sedensenet.py:111: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [7]:
densenet = models.densenet161(pretrained=True)
model_dict = model.state_dict()
densenet161_pretrained_dict = densenet.state_dict()
densenet161_pretrained_dict = {k: v for k, v in densenet161_pretrained_dict.items() if k in model_dict}
model_dict.update(densenet161_pretrained_dict)
model.load_state_dict(model_dict)
print("model_dict loaded.")

/home/usst/anaconda3/envs/dxzpy/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


model_dict loaded.


In [8]:
for para in list(model.parameters()):
    para.requires_grad=False
for para in list(model.features.denseblock3.parameters()):
    para.requires_grad=True
for para in list(model.features.transition3.parameters()):
    para.requires_grad=True
for para in list(model.features.denseblock4.parameters()):
    para.requires_grad=True
for para in list(model.features.norm5.parameters()):
    para.requires_grad=True

for para in list(model.sqexblock1.parameters()):
    para.requires_grad=True
for para in list(model.sqexblock2.parameters()):
    para.requires_grad=True
for para in list(model.sqexblock3.parameters()):
    para.requires_grad=True
for para in list(model.sqexblock4.parameters()):
    para.requires_grad=True
for para in list(model.sqextrans1.parameters()):
    para.requires_grad=True
for para in list(model.sqextrans2.parameters()):
    para.requires_grad=True
for para in list(model.sqextrans3.parameters()):
    para.requires_grad=True

In [9]:
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2208, num_classes),
)

In [10]:
device_ids = [0,1]

if use_gpu:
    model = model.cuda(device_ids[0])
    model = nn.DataParallel(model, device_ids=device_ids)
model.load_state_dict(torch.load('sedensenet.pth'))

In [11]:
class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        label,img_name = self.df[index]
        img_path = os.path.join(self.data_dir, img_name)
        with Image.open(img_path) as img:
            image = img.convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [12]:
batch_size = 64

trans_train = transforms.Compose([transforms.RandomResizedCrop(size=224),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomRotation(30),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.47954108864506007, 0.5295650244021952, 0.39169756009537665],
                                                       std=[0.21481591229053462, 0.20095268035289796, 0.24845895286079178])])

trans_valid = transforms.Compose([transforms.Resize(size=256),
                                  transforms.CenterCrop(size=224),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.47954108864506007, 0.5295650244021952, 0.39169756009537665],
                                                       std=[0.21481591229053462, 0.20095268035289796, 0.24845895286079178])])

dataset_train = MyDataset(df_data=train, 
    data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, 
    data_dir=val_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=False, num_workers=0)

In [13]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 module.features.denseblock3.denselayer1.norm1.weight
	 module.features.denseblock3.denselayer1.norm1.bias
	 module.features.denseblock3.denselayer1.conv1.weight
	 module.features.denseblock3.denselayer1.norm2.weight
	 module.features.denseblock3.denselayer1.norm2.bias
	 module.features.denseblock3.denselayer1.conv2.weight
	 module.features.denseblock3.denselayer2.norm1.weight
	 module.features.denseblock3.denselayer2.norm1.bias
	 module.features.denseblock3.denselayer2.conv1.weight
	 module.features.denseblock3.denselayer2.norm2.weight
	 module.features.denseblock3.denselayer2.norm2.bias
	 module.features.denseblock3.denselayer2.conv2.weight
	 module.features.denseblock3.denselayer3.norm1.weight
	 module.features.denseblock3.denselayer3.norm1.bias
	 module.features.denseblock3.denselayer3.conv1.weight
	 module.features.denseblock3.denselayer3.norm2.weight
	 module.features.denseblock3.denselayer3.norm2.bias
	 module.features.denseblock3.denselayer3.conv2.weight
	 module.features.dens

In [14]:
num_epochs = 10
early_stopping = 5

In [15]:
def cross_entropy(input, target, size_average=True):
    """ Cross entropy that accepts soft targets
    Args:
         pred: predictions for neural network
         targets: targets, can be soft
         size_average: if false, sum is returned instead of mean

    Examples::

        input = torch.FloatTensor([[1.1, 2.8, 1.3], [1.1, 2.1, 4.8]])
        input = torch.autograd.Variable(out, requires_grad=True)

        target = torch.FloatTensor([[0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])
        target = torch.autograd.Variable(y1)
        loss = cross_entropy(input, target)
        loss.backward()
    """
    logsoftmax = nn.LogSoftmax()
    if size_average:
        return torch.mean(torch.sum(-target * logsoftmax(input), dim=1))
    else:
        return torch.sum(torch.sum(-target * logsoftmax(input), dim=1))

In [16]:
criterion = cross_entropy
optimizer = optim.Adam(params_to_update,lr = 1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
label_smoothing = 0.1

best_val_acc = 0.86
best_epoch = 0
epoch_since_best = 0

for epoch in range(num_epochs):
    at = time.time()
    scheduler.step()
    model.module.train()
    train_total_samples = 0
    train_acc = 0
    train_loss = 0
    for i, data in enumerate(loader_train):
        print('.',end='')        
        inputs, label = data
        train_total_samples += label.size()[0]        
        labels = label.resize(label.size()[0], 1)
        labels = torch.FloatTensor(label.size()[0], n_class).zero_().scatter_(1, labels.resize(label.size()[0], 1) ,1)
        labels = (1 - label_smoothing) * labels + (label_smoothing / n_class)
        if use_gpu:
            inputs, labels, label = inputs.cuda(), labels.cuda(), label.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_pred = torch.argmax(outputs.data, dim=1)        
        train_acc += torch.sum(train_pred == label.data)
        train_loss += loss.item() * labels.size()[0]
            
    model.module.eval()
    valid_total_samples = 0
    valid_acc = 0
    val_loss = 0
    for _, data in enumerate(loader_valid):     
        inputs, label = data
        valid_total_samples += label.size()[0]        
        labels = label
        labels = torch.FloatTensor(label.size()[0], n_class).zero_().scatter_(1, labels.resize(label.size()[0], 1) ,1)
        labels = (1 - label_smoothing) * labels + (label_smoothing / n_class)
        if use_gpu:
            inputs, labels, label = inputs.cuda(), labels.cuda(), label.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        valid_pred = torch.argmax(outputs.data, dim=1)        
        valid_acc += torch.sum(valid_pred == label.data)
        val_loss += loss.item() * labels.size()[0]

    train_acc = train_acc.cpu().numpy() / train_total_samples
    valid_acc = valid_acc.cpu().numpy() / valid_total_samples
    train_loss = train_loss / train_total_samples
    val_loss = val_loss / valid_total_samples
    
    print()
    bt = time.time()
    print('[Epoch %d] train loss %.6f train acc %.6f  valid loss %.6f valid acc %.6f  time %.6f' % (
        epoch, train_loss, train_acc, val_loss, valid_acc,bt-at))

    if valid_acc > best_val_acc:
        best_val_acc = valid_acc
        best_epoch = epoch
        epoch_since_best = 0
        print('save model...')
        torch.save(model.state_dict(), 'sedensenet.pth')
        print('saved.')
    else:
        epoch_since_best += 1
        
    if epoch_since_best > early_stopping:
        break
            
print('Finished Training')
print('best_epoch: %d, best_val_acc %.6f' % (best_epoch, best_val_acc))

.

/home/usst/anaconda3/envs/dxzpy/lib/python3.6/site-packages/torch/tensor.py:263: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/usst/anaconda3/envs/dxzpy/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
[Epoch 0] train loss 1.056244 train acc 0.857494  valid loss 1.028638 valid acc 0.864978  time 762.756546
save model...
saved.
.........................................................................................................................................................................................................................................................................................................................................................................................

In [19]:
criterion = cross_entropy
optimizer = optim.SGD(params_to_update,lr = 1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
label_smoothing = 0.1

best_val_acc = 0.874449
best_epoch = 6
epoch_since_best = 0